In [1]:
import pandas as pd
import os
import time

import torch
from tqdm import tqdm
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.
from openai import OpenAI # openai==1.2.0
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain_community.llms import Ollama

In [2]:
tokenizer = AutoTokenizer.from_pretrained("yanolja/EEVE-Korean-10.8B-v1.0")
model = AutoModelForCausalLM.from_pretrained(
    "yanolja/EEVE-Korean-10.8B-v1.0",
    device_map="auto",
    torch_dtype=torch.float16,
)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [38]:
# 데이터 경로를 지정해줍니다.
DATA_PATH = "../data/"
RESULT_PATH = "./prediction/"

# train data의 구조와 내용을 확인합니다.
train_df = pd.read_csv(os.path.join(DATA_PATH,'train_all.csv'))
train_df.tail()

,fname,ground_dialogue_ko,dialogue_en_trans,dialogue_en_trans2ko,groundtruth_summary,solarAPI_summary_from_ground_dialogue,solarAPI_summary_from_dialogue_en_trans2ko,topic,NER
3996,train_3996,#Person1#: 제가 도와드릴 일이 있을까요?\n#Person2#: 유럽 여행을...,#Person1#: Is there anything I can help you wi...,#Person1#: 도와드릴 일이 있으신가요? \n#Person2#: 유럽 여행을...,#Person2#가 #Person1#에게 유럽 여행 경로의 가격에 대해 묻고 있습니다.,"#Person1#이 여행 경로 가격을 문의하자, #Person2#는 400달러에 4...","#Person1#: 유럽 여행 계획 관련 문의를 받고, 방문 도시 확정 여부를 확인...",유럽 여행 비용,Named Entities: 유럽
3997,train_3997,#Person1#: 주문하시겠어요?\n#Person2#: 버거 하나랑 큰 감자튀김 ...,#Person1#: Are you ready to order? \n#Person2...,#Person1#: 주문하시겠어요? \n#Person2#: 버거 하나와 라지 프라...,"고객이 버거, 큰 감자튀김, 큰 콜라를 주문합니다.","#Person1#이 주문을 요청하자, #Person2#는 버거 1개와 큰 감자튀김,...","#Person2#가 버거 1개, 라지 프라이, 라지 Coke를 주문하고 매장에서 식...",버거 주문,"버거, 감자튀김, 콜라"
3998,train_3998,"#Person1#: Crabby 씨, 만나 뵙게 되어 반갑습니다.\n#Person2...","#Person1#: Nice to meet you, Mr. Crabby. \n#P...","#Person1#: 크래비 씨, 만나서 반갑습니다. \n#Person2#: 안녕하...",Crabby 씨는 Jane 씨를 면접하면서 이전 회사에서의 능력에 대해 질문합니다....,Jane은 Crabby와의 면접에서 대학 졸업 후 2년간 근무한 비서 경험을 언급하...,제인 씨는 대학 졸업 후 2년간 비서 업무를 수행하며 좋은 동료들과 즐겁게 근무했으...,비서 경험 이직 이유,"Jane, Crabby"
3999,train_3999,"#Person1#: 루시, 이 편지들 좀 보내줄래? \n#Person2#: 네, 알...","#Person1#: Lucy, could you send these letters ...","#Person1#: 루시, 이 편지들을 대신 보내줄 수 있나요? \n#Person...",#Person1#은 #Person2#에게 런던에 있는 Mr. Charles에게 급한...,#Person1#이 루시에게 Mr. Charles에게 보낼 편지들을 처리하도록 지시...,#Person1#이 루시에게 찰스 씨에게 보낼 편지 처리에 대해 지시함. 두 통은 ...,우편 발송 요청,"루시, Mr. Charles, 런던"
4000,train_4000,"#Person1#: 여보세요, 거기 계세요?\n#Person2#: 네, 여기 있습니...",#Person1#: Hello? Are you there? \n#Person2#:...,"#Person1#: 여보세요? 거기 계세요? \n#Person2#: 네, 여기 있...",#Person1#은 전화로 보안 목적으로 Mike Kowalski에게 몇 가지 질문...,#Person1#이 #Person2#에게 호텔 리셉션에 분실물 접수 여부를 확인한 ...,#Person1#이 #Person2#에게 호텔 리셉션에 제출된 정보가 없음을 확인한...,호텔 보안 확인,"Mike Kowalski, #CardNumber#, #DateOfBirth#"


## 1. Solar Chat API 요약 성능 확인하기
- Solar Chat API을 이용하여 train 및 validation dataset에 포함된 dialogue 샘플을 요약해 봅니다.

In [69]:
# 모델 성능에 대한 평가 지표를 정의합니다. 본 대회에서는 ROUGE 점수를 통해 모델의 성능을 평가합니다.
rouge = Rouge()
def compute_metrics(pred, gold):
    results = rouge.get_scores(pred, gold, avg=True)
    result = {key: value["f"] for key, value in results.items()}
    return result

In [125]:
# Dialogue를 입력으로 받아, Solar Chat API에 보낼 Prompt를 생성하는 함수를 정의합니다.
def build_prompt(dialogue):

    system_prompt = "You are an expert in the field of dialogue summarization. Please summarize the \
        following dialogue to a single short sentence within 100 tokens in English."
    return f"{system_prompt}\n\nDialogue:\n{dialogue}\n\nSummary:\n"


In [154]:
def build_tr_prompt(summary):

    system_prompt = "You are an expert in the field of translation. Please translate the \
        following English sentence to a single short Korean sentence."
    return f"{system_prompt}\n\nsentence:\n{summary}\n\ntranslation:\n"

In [143]:
def summarization(dialogue):
    # 프롬프트 생성: 사용자 정의 함수. 아래와 같이 simple template 가능
    prompt = build_prompt(dialogue)  # 예: f"다음 대화를 요약해줘:\n{dialogue}"
    
    # 토크나이즈 및 텐서화
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # Generation
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.1,
            top_p=0.3,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    # 출력 추출(프롬프트 부분 제외)
    generated = output[:, inputs.input_ids.shape[1]:]
    summary = tokenizer.decode(generated[0], skip_special_tokens=True)
    
    return summary.strip()


In [155]:
def translation(summary):
    # 프롬프트 생성: 사용자 정의 함수. 아래와 같이 simple template 가능
    prompt = build_tr_prompt(summary)  # 예: f"다음 대화를 요약해줘:\n{dialogue}"
    
    # 토크나이즈 및 텐서화
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # Generation
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.1,
            top_p=0.3,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    # 출력 추출(프롬프트 부분 제외)
    generated = output[:, inputs.input_ids.shape[1]:]
    summary = tokenizer.decode(generated[0], skip_special_tokens=True)
    
    return summary.strip()


In [151]:
result2 = compute_metrics(train_df['solarAPI_summary_from_ground_dialogue'], train_df['groundtruth_summary'])
print(result2)

{'rouge-1': 0.17895477530234521, 'rouge-2': 0.04690049597540933, 'rouge-l': 0.16967629058083766}


In [156]:
# Train data 중 처음 3개의 대화를 요약합니다.
def test_on_train_data(num_samples=3):
    start_time = time.time()
    for idx, row in train_df[:num_samples].iterrows():
        dialogue = row['ground_dialogue_ko']
        summary = summarization(dialogue)

        # Rate limit 방지를 위해 1분 동안 최대 100개의 요청을 보내도록 합니다.
        if (idx + 1) % 100 == 0:
            end_time = time.time()
            elapsed_time = end_time - start_time
            
            if elapsed_time < 60:
                wait_time = 60 - elapsed_time + 5
                print(f"Elapsed time: {elapsed_time:.2f} sec")
                print(f"Waiting for {wait_time} sec")
                time.sleep(wait_time)
            
            start_time = time.time()
            
        train_df.loc[idx, 'sentence_summary'] = summary
        convert = translation(summary)
        train_df.loc[idx, 'summary_from_en_dialoge2ko'] = convert

        # print(f"Dialogue:\n{dialogue}\n")
        print(f"Pred Summary: {summary}\n")
        print(f"Pred Summary2: {convert}\n")
        # print(f"Gold Summary: {row['groundtruth_summary']}\n")
        # print("=="*50)
    result = compute_metrics(train_df['summary_from_en_dialoge2ko'], train_df['groundtruth_summary'])
    print(result)

In [157]:
if __name__ == "__main__":
    test_on_train_data()

Pred Summary: The patient, Mr. Smith, visited the doctor for a health checkup after not having one for five years. The doctor advised him to get one every year and warned him about the dangers of smoking.

Pred Summary2: 환자, 스미스 씨는 5년 동안 건강 검진을 받지 않은 후 건강 검진을 위해 의사를 방문했습니다. 의사는 그에게 매년 건강 검진을 받으라고 조언하고 흡연의 위험에 대해 경고했습니다.

Pred Summary: Dr. Peters는 Ricky가 소아마비, 파상풍, B형 간염 예방접종을 받았으며, 현재 14개월이므로 A형 간염, 수두, 홍역 예방접종을 받아야 한다고 말했습니다. Mrs. Parker는 풍진과 유행성이하선염 예방접종도 필요하다고 말했고, Dr. Peters는 몇 주 후에 할 수 있다고 말했습니다. Mrs. Parker는 또한 파상풍 부스터가 필요하다고 말했고, Dr

Pred Summary2: Dr. Peters는 Ricky가 소아마비, 파상풍, B형 간염 예방접종을 받았으며, 현재 14개월이므로 A형 간염, 수두, 홍역 예방접종을 받아야 한다고 말했습니다. Mrs. Parker는 풍진과 유행성이하선염 예방접종도 필요하다고 말했고, Dr. Peters는 몇 주 후에 할 수 있다고 말했습니다. Mrs. Parker는 또한 파상풍 부스터가 필요하다고 말했고, Dr

Pred Summary: The person is looking for a specific set of keys with five keys and a small ball decoration, and asks for help from another person. The other person offers to help and eventually finds the keys, to which the person

ValueError: Hypothesis is empty.

Validation Dataset을 이용하여 요약을 진행하고, 성능을 평가해 봅니다.

In [15]:
# Validation data의 대화를 요약하고, 점수를 측정합니다.
def validate(num_samples=-1):
    val_samples = val_df[:num_samples] if num_samples > 0 else val_df
    
    scores = []
    for idx, row in tqdm(val_samples.iterrows(), total=len(val_samples)):
        dialogue = row['dialogue']
        summary = summarization(dialogue)
        results = compute_metrics(summary, row['summary'])
        avg_score = sum(results.values()) / len(results)
        
        scores.append(avg_score)
        
    val_avg_score = sum(scores) / len(scores)

    print(f"Validation Average Score: {val_avg_score}")

In [13]:
if __name__ == "__main__":
    validate(100) # 100개의 validation sample에 대한 요약을 수행합니다.
    
    # 전체 validation data에 대한 요약을 수행하고 싶은 경우 아래와 같이 실행합니다.
    # validate() 

100%|██████████| 100/100 [02:14<00:00,  1.34s/it]

Validation Average Score: 0.09606867456611186


# back translation

In [ ]:
# Train data 중 처음 3개의 대화를 요약합니다.
def test_on_train_data(num_samples):
    for idx, row in train_df[:num_samples].iterrows():
        dialogue = row['dialogue']
        summary = summarization(dialogue)
        print(f"Dialogue:\n{dialogue}\n")
        print(f"Dialogue:\n{summary}\n")
        print(f"Gold Summary: {row['summary']}\n")

        output_df = pd.DataFrame(
            {
                "fname": train_df['fname'],
                "dialogue": summary,
                "summary": row["summary"],
            }
        )

        if not os.path.exists(RESULT_PATH):
            os.makedirs(RESULT_PATH)
        output_df.to_csv(os.path.join(RESULT_PATH, "back_translation.csv"), index=False)

    return output_df



In [37]:
if __name__ == "__main__":
    back_translation = test_on_train_data(len(train_df))

Dialogue:
#Person1#: 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다. 오늘 무슨 일로 오셨어요? 
#Person2#: 건강검진을 받으려고 왔어요. 
#Person1#: 네, 5년 동안 검진을 안 받으셨네요. 매년 한 번씩 받으셔야 해요. 
#Person2#: 알죠. 특별히 아픈 데가 없으면 굳이 갈 필요가 없다고 생각했어요. 
#Person1#: 음, 심각한 질병을 피하려면 미리 발견하는 게 제일 좋거든요. 본인을 위해서라도 매년 한 번은 오세요. 
#Person2#: 알겠습니다. 
#Person1#: 여기 좀 볼까요. 눈과 귀는 괜찮으시네요. 깊게 숨 한 번 쉬어보세요. Mr. Smith, 담배 피우세요? 
#Person2#: 네. 
#Person1#: 담배가 폐암하고 심장병의 주된 원인인 거 아시죠? 끊으셔야 해요. 
#Person2#: 수백 번 시도했는데, 도저히 습관이 안 끊어져요. 
#Person1#: 음, 도움 될만한 수업과 약물들이 있습니다. 가시기 전에 더 정보를 드릴게요. 
#Person2#: 네, 고맙습니다, 의사 선생님.

Pred Summary: #Person1#: Hello, Mr. Smith. I'm Dr. Hawkins. What can I help you with today? 
#Person2#: I'm here for a health checkup. 
#Person1#: Yes, you haven't had one in five years. You should get one every year. 
#Person2#: I know. I just thought there was no need to go if I didn't feel sick. 
#Person1#: Well, it's better to catch serious diseases early. For your own sake, come every year. 
#Person2#: Okay. 
#Person1#: Let's see. Your e

KeyboardInterrupt: 

## 2. Solar Chat API로 요약하기
- Solar Chat API을 이용하여 test dataset에 포함된 dialogue를 요약하고 제출용 파일을 생성합니다.

In [12]:
def inference():
    test_df = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))

    summary = []
    start_time = time.time()
    for idx, row in tqdm(test_df.iterrows(), total=len(test_df)):
        dialogue = row['dialogue']
        summary.append(summarization(dialogue))
        
        # Rate limit 방지를 위해 1분 동안 최대 100개의 요청을 보내도록 합니다.
        if (idx + 1) % 100 == 0:
            end_time = time.time()
            elapsed_time = end_time - start_time
            
            if elapsed_time < 60:
                wait_time = 60 - elapsed_time + 5
                print(f"Elapsed time: {elapsed_time:.2f} sec")
                print(f"Waiting for {wait_time} sec")
                time.sleep(wait_time)
            
            start_time = time.time()
    
    output = pd.DataFrame(
        {
            "fname": test_df['fname'],
            "summary" : summary,
        }
    )
    
    if not os.path.exists(RESULT_PATH):
        os.makedirs(RESULT_PATH)
    output.to_csv(os.path.join(RESULT_PATH, "output_yanolja.csv"), index=False)

    return output

In [13]:
if __name__ == "__main__":
    output = inference()

  0%|          | 0/499 [00:00<?, ?it/s]

100%|██████████| 499/499 [1:03:29<00:00,  7.64s/it]


In [14]:
output  # 각 대화문에 대한 요약문이 출력됨을 확인할 수 있습니다.

,fname,summary
0,test_0,Ms. Dawson will write a memo to be distributed...
1,test_1,Person2 is late because of traffic on the way ...
2,test_2,Masha and Hero are getting divorced. Masha wil...
3,test_3,Person1 and Person2 are at a party for Person2...
4,test_4,The Olympic Park is huge and the stadium is be...
...,...,...
494,test_495,Jack invites Charlie to play a new game he bou...
495,test_496,Person2 started listening to country music aft...
496,test_497,Person1 is new to the laundry room and asks Pe...
497,test_498,#Person1# and #Person2# haven't seen each othe...


# translation

In [ ]:
def translation():
    test_df = pd.read_csv(os.path.join(DATA_PATH, 'output_yanolja.csv'))
    sentences = []

    pad_token = tokenizer.eos_token_id if tokenizer.eos_token_id is not None else model.config.pad_token_id

    for row in tqdm(test_df.itertuples(index=False), total=len(test_df)):
        summary_text = row.summary

        prompt = f"Please translate the following English sentence into Korean. Person names should be kept in English.\n sentence:\n{summary_text}\n\n translation:\n"
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_new_tokens=256,
                temperature=0.1,
                top_p=0.3,
                do_sample=True,
                pad_token_id=pad_token,
            )
            generated = output[:, inputs.input_ids.shape[1]:]
            translation = tokenizer.decode(generated[0], skip_special_tokens=True)
            # print(translation)
            sentences.append(translation)

        # Optional: torch.cuda.empty_cache()

    output_df = pd.DataFrame(
        {
            "fname": test_df['fname'],
            "summary": sentences,
        }
    )

    if not os.path.exists(RESULT_PATH):
        os.makedirs(RESULT_PATH)
    output_df.to_csv(os.path.join(RESULT_PATH, "output_yanolja_tr.csv"), index=False)

    return output_df


In [21]:
if __name__ == "__main__":
    output = translation()

  0%|          | 1/499 [00:03<33:03,  3.98s/it]

Ms. Dawson은 오늘 오후 4시까지 모든 직원에게 배포될 메모를 작성하여, 모든 내부 및 외부 커뮤니케이션은 이메일이나 공식 메모를 통해 이루어져야 하며, 근무 시간 동안에는 인스턴트 메시징의 사용이 금지된다고 명시할 것입니다.


  0%|          | 2/499 [00:09<38:08,  4.60s/it]

Person2는 Person1의 집으로 가는 길에 교통 체증 때문에 늦었습니다. Person1은 다른 경로를 택하거나 대중 교통을 이용하는 것을 제안하고, Person2는 이를 고려하기로 동의합니다. 그들은 대중 교통의 이점과 운전으로 제공되는 자유에 대해 논의합니다. Person2는 운전 대신 자전거로 출근하기로 결정합니다.


  1%|          | 3/499 [00:10<27:09,  3.29s/it]

...

Person1은 세탁실에 새로 왔고 Person2에게 세탁기 사용 방법을 묻습니다. Person2는 큰 세탁기는 건조기이고 작은 세탁기는 세탁기라고 설명합니다. Person1은 세탁기에 비누가 있는지 묻고, Person2는 따로 추가해야 한다고 말합니다. Person1은 비누가 없다고 말하고, Person2는 비누를 어디서 살 수 있는지 알려줍니다. Person1은 고맙다고 말하고 많은 양의 비누를 샀다고 말하고, Person2는 너무 많이 사용하면 세균이 번식할 수 있다고 경고합니다. Person1은 세탁을 해본 적이 없고 엄마가 항상 해줬다고 말합니다. Person2는 12살 때부터 스스로 세탁을 했다고 말하고, Person1은 미국 아이들은 더 독립적이지만 엄마가 좋은 성적을 받기 위해 모든 것을 해줬다고 말합니다. Person2는 캠퍼스에서 어떻게 지낼지 묻고, Person1은 모르겠지만 배워야 한다고 말합니다.


100%|█████████▉| 498/499 [42:44<00:11, 11.53s/it]

#Person1#과 #Person2#는 1년 동안 서로 만나지 못했습니다. #Person1#의 현재 임대 계약이 다음 달에 만료되므로 #Person1#은 새로운 거주지를 찾고 있습니다. #Person2#는 이웃의 아파트를 살펴볼 것을 제안합니다. 이웃의 딸이 함께 살기로 결정했기 때문에 아파트가 비어 있습니다. #Person1#은 관심이 있고 #Person2#는 더 많은 정보를 #Person1#에게 알려줄 것입니다.


100%|██████████| 499/499 [42:46<00:00,  5.14s/it]

Frank가 승진하고 토요일에 150명의 사람들을 위한 파티를 열 예정입니다. Betsy는 초대되었습니다.


In [26]:
import csv
import re

# 입력/출력 파일 경로
input_file = '/data/ephemeral/home/dev/code/outout_yanolja.csv'
output_file = '/data/ephemeral/home/dev/code/outout_yanolja_clean.csv'

def clean_text(text):
    # 기본 전처리: 양쪽 공백 제거, 줄바꿈 통일, 이중 공백 1개로
    text = text.strip()
    text = text.replace('\r\n', '\n').replace('\r', '\n')  # 윈도우/맥 줄바꿈 통일
    text = re.sub(r'\n+', ' ', text)  # 줄바꿈을 공백 한 칸으로 변경
    text = re.sub(r'\s+', ' ', text)  # 다중 공백을 단일 공백으로 변경

    # 추가 정리 (필요시 확장 가능)
    # 예) 불필요한 특수문자 제거 등
    # text = re.sub(r'[^\S\r\n]+', ' ', text)

    return text

def main():
    with open(input_file, 'r', encoding='utf-8-sig') as rf, \
         open(output_file, 'w', encoding='utf-8', newline='') as wf:
        
        reader = csv.DictReader(rf)
        fieldnames = reader.fieldnames
        writer = csv.DictWriter(wf, fieldnames=fieldnames)
        writer.writeheader()

        for row in reader:
            # 텍스트 정리 (summary 칼럼)
            if 'summary' in row:
                row['summary'] = clean_text(row['summary'])
            writer.writerow(row)

    print(f"파일 정리가 완료되어 {output_file}로 저장되었습니다.")

if __name__ == '__main__':
    main()


파일 정리가 완료되어 /data/ephemeral/home/dev/code/outout_yanolja_clean.csv로 저장되었습니다.


In [30]:
df = pd.read_csv("/data/ephemeral/home/dev/code/outout_yanolja_clean.csv")

## 3. Prompt Engineering
- Prompt engineering을 통해 요약 성능 향상을 시도합니다.

In [ ]:
# Few-shot prompt를 생성하기 위해, train data의 일부를 사용합니다.
few_shot_samples = train_df.sample(1)

sample_dialogue1 = few_shot_samples.iloc[0]['dialogue']
sample_summary1 = few_shot_samples.iloc[0]['summary']

print(f"Sample Dialogue1:\n{sample_dialogue1}\n")
print(f"Sample Summary1: {sample_summary1}\n")

In [ ]:
# Prompt를 생성하는 함수를 수정합니다.
def build_prompt(dialogue):
    system_prompt = "You are a expert in the field of dialogue summarization, summarize the given dialogue in a concise manner. Follow the user's instruction carefully and provide a summary that is relevant to the dialogue."

    user_prompt = (
        "Following the instructions below, summarize the given document.\n"
        "Instructions:\n"
        "1. Read the provided sample dialogue and corresponding summary.\n"
        "2. Read the dialogue carefully.\n"
        "3. Following the sample's style of summary, provide a concise summary of the given dialogue.\n\n"
        "Sample Dialogue:\n"
        f"{sample_dialogue1}\n\n"
        "Sample Summary:\n"
        f"{sample_summary1}\n\n"
        "Dialogue:\n"
        f"{dialogue}\n\n"
        "Summary:\n"
    )
    
    return [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": user_prompt
        }
    ]

In [ ]:
# 변경된 prompt를 사용하여, train data 중 처음 3개의 대화를 요약하고, 결과를 확인합니다.
if __name__ == "__main__":
    test_on_train_data()

In [ ]:
# 변경된 prompt를 사용하여, validation data의 대화를 요약하고, 점수를 측정합니다.
if __name__ == "__main__":
    validate(100)

다른 방식으로 Few-shot sample을 제공하여 Prompt를 구성해 봅니다.

In [ ]:
# Few-shot sample을 다른 방식으로 사용하여 prompt를 생성합니다.
def build_prompt(dialogue):
    system_prompt = "You are a expert in the field of dialogue summarization, summarize the given dialogue in a concise manner. Follow the user's instruction carefully and provide a summary that is relevant to the dialogue."

    few_shot_user_prompt_1 = (
        "Following the instructions below, summarize the given document.\n"
        "Instructions:\n"
        "1. Read the provided sample dialogue and corresponding summary.\n"
        "2. Read the dialogue carefully.\n"
        "3. Following the sample's style of summary, provide a concise summary of the given dialogue. Be sure that the summary is simple but captures the essence of the dialogue.\n\n"
        "Dialogue:\n"
        f"{sample_dialogue1}\n\n"
        "Summary:\n"
    )
    
    few_shot_assistant_prompt_1 = sample_summary1
    
    user_prompt = (
        "Dialogue:\n"
        f"{dialogue}\n\n"
        "Summary:\n"
    )
    
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": few_shot_user_prompt_1},
        {"role": "assistant", "content": few_shot_assistant_prompt_1},
        {"role": "user", "content": user_prompt},
    ]

In [ ]:
# 변경된 prompt를 사용하여, train data 중 처음 3개의 대화를 요약하고, 결과를 확인합니다.
if __name__ == "__main__":
    test_on_train_data()

In [ ]:
# 변경된 prompt를 사용하여, validation data의 대화를 요약하고, 점수를 측정합니다.
if __name__ == "__main__":
    validate(100)

### (선택) 변경된 Prompt로 test dataset에 대한 요약을 진행합니다.
- 변경된 prompt를 통해 점수가 개선되었다면, test dataset에 대한 요약을 진행하고 제출합니다.

In [ ]:
# 변경된 prompt를 사용하여, test data의 대화를 요약하고, 결과를 확인합니다.
if __name__ == "__main__":
    output = inference()

In [ ]:
output